In [ ]:
!apt-get install tesseract-ocr -y
!pip install --upgrade --force-reinstall transformers
!pip install pytesseract transformers torch pillow


In [4]:
from google.colab import files
uploaded = files.upload()  # Select receipts.db


Saving db.ipynb to db.ipynb


In [9]:
import pytesseract
from PIL import Image
from transformers import pipeline
from datetime import datetime
from db import insert_receipt



#LLM pipeline
classifier = pipeline("text2text-generation", model="google/flan-t5-small")






Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  sqlite3-doc
The following NEW packages will be installed:
  sqlite3
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 769 kB of archives.
After this operation, 1,874 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sqlite3 amd64 3.37.2-2ubuntu0.5 [769 kB]
Fetched 769 kB in 0s (2,400 kB/s)
Selecting previously unselected package sqlite3.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../sqlite3_3.37.2-2ubuntu0.5_amd64.deb ...
Unpacking sqlite3 (3.37.2-2ubuntu0.5) ...
Setting up sqlite3 (3.37.2-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [10]:
def extract_text(image_path):
    """Runs OCR on the receipt image."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text.strip()

In [11]:
def classify_cat(text):
    """Uses FLAN-T5 to classify receipt text into a category."""
    prompt = f"""Classify the following expense text into one of these categories: Groceries, Food, Transport, Healthcare, Shopping, Entertainment, Utilities, Other.
    Text: {text}
    Category:"""       #llm fills in the blank.
    result = classifier(prompt, max_length=10, do_sample=False)[0]['generated_text']   #returns list with one dict. do sample (f) returns deterministic ans.
    return result.strip()


In [12]:
def extract_amount(text):
    """Extracts the largest floating point number assuming it's the total amount."""
    import re
    amounts = re.findall(r'\d+\.\d{2}', text)
    if amounts:
        return float(max(amounts, key=lambda x: float(x)))
    return None

In [13]:
def process_receipt(image_path, date_str):
    """Main pipeline: OCR + classification + DB insertion."""
    print(f"Processing {image_path}...")
    raw_text = extract_text(image_path)
    category = classify_cat(raw_text)
    amount = extract_amount(raw_text)
    date = datetime.strptime(date_str, "%Y-%m-%d").date()

    print("Extracted:", raw_text[:100], "...")
    print("Category:", category)
    print("Amount:", amount)
    print("Date:", date)

    if amount is not None:
        insert_receipt(raw_text, category, date.isoformat(), amount)
        print("Inserted into DB.")
    else:
        print("Amount not found. Skipping DB insert.")

In [21]:
# uploading sample img and using all func manually
uploaded = files.upload()
filename = list(uploaded.keys())[0]
text = extract_text(filename)
cat=classify_cat(text)
amt=extract_amount(text)
date=datetime.now().date()
insert_receipt(text, cat, date, amt) #upload to db

Saving illustration-receipt-template-black-white-vector-129915676.webp to illustration-receipt-template-black-white-vector-129915676 (7).webp


Both `max_new_tokens` (=256) and `max_length`(=10) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
